In [2]:
import unittest
import os
import pytest
from aes256_ctr_drbg import AES256_CTR_DRBG

from kyber import Kyber

DEFAULT_PARAMETERS = {
    "kyber_512": {
        "k": 2,
        "eta_1": 3,
        "eta_2": 2,
        "du": 10,
        "dv": 4,
    },
    "kyber_768": {
        "k": 3,
        "eta_1": 2,
        "eta_2": 2,
        "du": 10,
        "dv": 4,
    },
    "kyber_1024": {
        "k": 4,
        "q": 3329,
        "eta_1": 2,
        "eta_2": 2,
        "du": 11,
        "dv": 5,
    },
}
"""
Lattice parameters for the Kyber key exchange.

To be used for initialisation of :py:obj:`.Kyber` objects.
"""

# Initialise with default parameters for easy import
Kyber512 = Kyber(DEFAULT_PARAMETERS["kyber_512"])
"""
Key exchange object that uses Kyber512 parameters internally.

Provides about 128 bit level of security.
"""

Kyber768 = Kyber(DEFAULT_PARAMETERS["kyber_768"])
"""
Key exchange object that uses Kyber768 parameters internally.

Provides about 192 bit level of security.
"""

Kyber1024 = Kyber(DEFAULT_PARAMETERS["kyber_1024"])
"""
Key exchange object that uses Kyber1024 parameters internally.

Provides about 256 bit level of security.
"""



def parse_kat_data(data):
    parsed_data = {}
    count_blocks = data.split("\n\n")
    for block in count_blocks[:-1]:
        block_data = block.split("\n")
        count, seed, pk, sk, ct, ss = [
            line.split(" = ")[-1] for line in block_data
        ]
        parsed_data[int(count)] = {
            "seed": bytes.fromhex(seed),
            "pk": bytes.fromhex(pk),
            "sk": bytes.fromhex(sk),
            "ct": bytes.fromhex(ct),
            "ss": bytes.fromhex(ss),
        }
    return parsed_data


class TestKyber(unittest.TestCase):
    """
    Test Kyber levels for internal consistency by generating keypairs and
    shared secrets.
    """

    def test_update(self,Kyber,count):
        for _ in range(count):
            pk, sk = Kyber.keygen()
            pk,sk,sigma=Kyber.upKey(pk,sk,0)
            i=0
            while(True):
                pk,sk,sigma=Kyber.upKey(pk,sk,sigma)
                key, c = Kyber.encaps(pk)
                # Correct decaps works
                _key = Kyber.decaps(sk, c)
                print(key==_key,key,_key)
                if key==_key:
                    i+=1
                    continue
                break
            print(i)
#                 # Incorrect ct does not work
#                 _bad_ct = bytes([0] * len(c))
#                 _bad = Kyber.decaps(sk, _bad_ct)
#                 self.assertNotEqual(key, _bad)



In [3]:
a=TestKyber()

In [4]:
a.test_update(Kyber1024, 1)

True b'\x8f\xc4X\xf0\xb6\xd5G4\xa1\xc7\x04\xca\xf3\xe0\xf1\x8dd\xbd}\x0fG2\xb2\xad\xfb\x03\x96\x07<\xbe\xc4:' b'\x8f\xc4X\xf0\xb6\xd5G4\xa1\xc7\x04\xca\xf3\xe0\xf1\x8dd\xbd}\x0fG2\xb2\xad\xfb\x03\x96\x07<\xbe\xc4:'
True b'\x98\x91\xd2\xce\xb7j\x8a\x00\xfa\xf7@ht@\xfc\xbe\xb9\x86 \r]\xa5\xb2\x03(\xb8\xf2\x14\x10\xb0\xff\xa2' b'\x98\x91\xd2\xce\xb7j\x8a\x00\xfa\xf7@ht@\xfc\xbe\xb9\x86 \r]\xa5\xb2\x03(\xb8\xf2\x14\x10\xb0\xff\xa2'
True b'\x92\xa8}l\x1e\x1a\x93\xabF\\2\rD\xe0\xa7th\xa6\x1f\xc9\x94\x13\xe8M%\x8b`\x0c\ni\x99\xf1' b'\x92\xa8}l\x1e\x1a\x93\xabF\\2\rD\xe0\xa7th\xa6\x1f\xc9\x94\x13\xe8M%\x8b`\x0c\ni\x99\xf1'
True b'0\x87IN\xa5\x9c9\xd3P\x90\xc2:*\xdb\xeb\xcb^\xd6\x86pG\xf3w\x89\xcaI\x85\xe4\x9f\x12\x17f' b'0\x87IN\xa5\x9c9\xd3P\x90\xc2:*\xdb\xeb\xcb^\xd6\x86pG\xf3w\x89\xcaI\x85\xe4\x9f\x12\x17f'
True b'\x9cH\xe3\x1a\xfeq\xccj\xbc\xd1\xd2\x8bY\x83\xcd\xe0\xe3\xea\x9d\x82\x99\x9b\x86\xfb\xc4\x19\xa0\xa6\xae^:\xe8' b'\x9cH\xe3\x1a\xfeq\xccj\xbc\xd1\xd2\x8bY\x83\xcd\xe0\xe3\xea\x9d\

KeyboardInterrupt: 